In [1]:
from SimpleHandEye.interfaces.cameras import RealSenseCamera
import cv2

In [2]:
def showImage(color_frame, depth_frame, ir1_frame, ir2_frame):
    cv2.imshow('image', color_frame)
    cv2.waitKey(33)

camera = RealSenseCamera(callback_fn=showImage)

intrinsics_params = camera.getIntrinsics()
K = intrinsics_params['RGB']['K']
D = intrinsics_params['RGB']['D']

# K,D

Enabling streams for camera:  141322250720


RuntimeError: start() cannot be called before stop()

In [ ]:
from SimpleHandEye.interfaces.apriltag import ApriltagTracker


tag_pose_tracker = ApriltagTracker(tag_size=0.172, # put your tag size here
                          intrinsic_matrix=K,
                          distortion_coeffs=D)

cam_T_tag = tag_pose_tracker.getPose(camera.color_frame, tag_id=0)

print(cam_T_tag)

In [ ]:
from SimpleHandEye.solvers import OpenCVSolver
import numpy as np
solver = OpenCVSolver()


In [ ]:
R = np.array([[ 1.00000000e+00 ,4.48086013e-13, -2.09999912e-10],
 [ 4.48086013e-13, -1.00000000e+00, -3.14018492e-16],
 [-2.09999912e-10, 3.14018398e-16, -1.00000000e+00]]) 

t = np.array([ 3.06890567e-01 ,-2.36797045e-16 , 5.90282052e-01])

base_T_hand = np.vstack([np.hstack([R, t.reshape(3,1)]),
                          np.array([0,0,0,1])])

print(base_T_hand)

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')

import FR3Py
FR3Py.__path__

import sys 
from FR3Py.robot.interface import FR3Real
robot = FR3Real()



In [ ]:
# import numpy as np
# import time
# from FR3Py.robot.model import PinocchioModel
# model = PinocchioModel()
# time.sleep(1)
# state = robot.getStates()
# q, dq = state['q'], state['dq']
# info = model.getInfo(q, dq)
# R, t = info['R_EE'], info['P_EE']

# base_T_hand = np.vstack([np.hstack([R, t.reshape(3,1)]),
#                         np.array([0,0,0,1])])
# base_T_hand

In [ ]:
import numpy as np
import time
from FR3Py.robot.model import PinocchioModel

model = PinocchioModel()
time.sleep(1)

def getHandPose(robot):
  state = robot.getStates()

  if state is not None:
    q, dq = state['q'], state['dq']
    info = model.getInfo(q, dq)
    R, t = info['R_HAND'], info['P_HAND']

    base_T_hand = np.vstack([np.hstack([R, t.reshape(3,1)]),
                            np.array([0,0,0,1])])
    return base_T_hand
  else:
    return None

base_T_hand = getHandPose(robot)

print(base_T_hand)

In [ ]:
import ipywidgets as widgets
import numpy as np
from IPython.display import display
from pprint import pprint
from IPython.display import clear_output
np.set_printoptions(suppress=True, precision=3)

# The dataset
A_list = []
B_list = []
apriltag_info = []  
apriltag_imgs_raw = []
apriltag_imgs_udist = []

def on_sample_clicked(b):
    A  = getHandPose(robot)
    img = camera.color_frame
    info = tag_pose_tracker.getPoseAndCorners(img, tag_id=0)
    B = np.linalg.inv(info['pose']) # Note that B is tag_T_cam
    apriltag_info.append(info)
    apriltag_imgs_raw.append(img)
    apriltag_imgs_udist.append(tag_pose_tracker.undistortImage(img))
    print("A=")
    pprint(A)
    print("B=")
    pprint(B)
    # if A is not None and B is not None:
    A_list.append(A)
    B_list.append(B)
    print("*************")

def on_compute_clicked(b):
    try:
        X,Y = solver.solve(A_list, B_list)
        clear_output(wait=True)
        print("X=")
        pprint(X)
        print("Y=")
        pprint(Y)
    except:
        print("Bad dataset, please record again")
        A_list.clear()
        B_list.clear()
        

sample_button = widgets.Button(description="Sample")
compute_button = widgets.Button(description="Compute")

sample_button.on_click(on_sample_clicked)
compute_button.on_click(on_compute_clicked)
display(sample_button)
display(compute_button)